In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance


from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.cluster import KMeans

In [6]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [7]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])

print(np.max(label_visiprog))

200.0


In [8]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)
# print(X_STSIMM)

In [20]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

N_material = 61
N_material_train = 10

for fold in range(10):
    print("Trial {}".format(fold))

    # randomly select N_material_train material for training the metric
    # and test on the rest
    train_select = np.zeros_like(label_material).astype(bool)

    idx_train = np.array([])
    for i in range(N_material_train):
        train_class = np.random.randint(0, N_material)
        train_select |= (label_material == train_class)

    idx_train = np.where(train_select == True)[0]
    idx_test = np.where(train_select == False)[0]

    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)

    print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

    X_train = raw_feature[idx_train,:]
    X_STSIMM_train = X_STSIMM[idx_train,:]
    Y_VSP_train = label_visiprog[idx_train]
    Y_mat_train = label_material[idx_train]

    X_test = raw_feature[idx_test,:]
    X_STSIMM_test = X_STSIMM[idx_test,:]
    Y_VSP_test = label_visiprog[idx_test]
    Y_mat_test = label_material[idx_test]

    res_STSIMM = leave_one_sample_out(X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res_STSIMM['accuracy'])
    print(res_STSIMM['prediction'])
    print(Y_mat_test)

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(NaN)

    model_vsp_original = LFDA()
    try:
        model_vsp_original.fit(X_train, Y_VSP_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(NaN)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(NaN)

Trial 0
Training shape 803 and test shape 4442
[18  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 1
Training shape 920 and test shape 4325
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 2
Training shape 860 and test shape 4385
[15  1  1 ..., 60 60 60]
[ 1  1  1 ..., 60 60 60]
Trial 3
Training shape 784 and test shape 4461
[17  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 4
Training shape 883 and test shape 4362
[24  1  1 ..., 60 60 60]
[ 1  1  1 ..., 60 60 60]
Trial 5
Training shape 914 and test shape 4331
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 6
Training shape 864 and test shape 4381
[15  1  1 ..., 59 59 58]
[ 1  1  1 ..., 59 59 59]
Trial 7
Training shape 807 and test shape 4438
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 8
Training shape 731 and test shape 4514
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 9
Training shape 920 and test shape 4325
[17  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]


In [30]:
print(len(accuracy_mat))
print(len(accuracy_vsp))
print(len(accuracy_STSIMM))
print(len(accuracy_vsp_original))

print(res_STSIMM['accuracy'])
print(accuracy_STSIMM)

10
10
10
10
0.880043033889
[0.88701110574601638, 0.87974345269909138, 0.91476323119777159, 0.88532707844670311, 0.87946777980693969, 0.88285184203551548, 0.8778484366719661, 0.88400323711896411, 0.89361118727721411, 0.8800430338891877]


In [21]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])

df.loc['average'] = df.mean()

display(df)
# display(df.mean())

,STSIMM,material,visiprog_original,visiprog
0,0.874381,0.970959,0.972310,0.975462
1,0.879306,0.976879,0.980578,0.965549
2,0.872064,0.978107,0.978563,0.972406
3,0.888142,0.973997,0.975342,0.977584
4,0.874140,0.970197,0.965383,0.966758
5,0.888940,0.979681,0.969060,0.971600
6,0.882903,0.955718,0.981739,0.958229
7,0.863227,0.957639,0.933979,0.946823
8,0.871954,0.959681,0.959238,0.951706
9,0.881850,0.971561,0.972254,0.968092


In [ ]:
print(model.transformer())